In [1]:
import pandas as pd
import requests
from bs4 import BeautifulSoup
import numpy as np
import yaml

In [3]:
try: 
    with open ("params.yaml", 'r') as file:
        config = yaml.safe_load(file)
except Exception as e:
    print('Error reading the config file')

{'data': {'hot_songs_file': 'hot100_songs.csv', 'not_hot_songs_file': 'song_data.csv'}, 'random_choice': {'seed': 2020, 'selection': 20000}}


In [4]:
def scrap_hot100(url):
    response = requests.get(url)
    soup = BeautifulSoup(response.content, 'html.parser')
    if (response.status_code == 200):   
        title = []
        artists = []
        for i in soup.select("li.o-chart-results-list__item > h3"):
            title.append(i.get_text().strip())
        for i in soup.select("li.o-chart-results-list__item > span:nth-child(2)"):
            artists.append(i.get_text().strip())
        unwanted = {'NEW', 'RE-\nENTRY'}
        artists = [ele for ele in artists if ele not in unwanted]
        songs = {'title': title,
                'artist': artists}
        hot100 = pd.DataFrame(songs).reset_index(drop = True)
        hot100.to_csv(config['data']['hot_songs_file'], index=False)
    return hot100

In [5]:
hot_songs = scrap_hot100('https://www.billboard.com/charts/hot-100/')
hot_songs.head()

,title,artist
0,Break My Soul,Beyonce
1,As It Was,Harry Styles
2,About Damn Time,Lizzo
3,Running Up That Hill (A Deal With God),Kate Bush
4,Staying Alive,DJ Khaled Featuring Drake & Lil Baby


# ------------------------------------------------------------------------ #

In [6]:
def NotHotSongs():

    #read hot file
    hot = pd.read_csv(config['data']['hot_songs_file'])
    #Reading the Kaggle Dataset
    songs = pd.read_csv(config['data']['not_hot_songs_file'])

    #Randomly select 20000 songs
    np.random.seed(config['random_choice']['seed'])
    row_index = np.random.choice(1116609, config['random_choice']['selection'])
    nothot = songs.iloc[row_index]

    #drop uneccessary columns and duplicates
    nothot.drop(['user_id', 'song_id', 'listen_count', 'song'], axis = 1, inplace = True)
    nothot.drop_duplicates(inplace = True)

    #Cleaning the Strings
    nothot['artist'] = nothot['artist'].str.replace('\\', '')
    nothot['title']= nothot['title'].str.replace('\\', '')
    nothot['artist']= nothot['artist'].str.replace('_', ',')
    nothot['title']= nothot['title'].str.replace('_', ',')

    words = ["radio", "Album", "album", "version", 
             "remix", "Version", "edit", "remix", "Remix", "acoustic", 
             "Acoustic", "@"]
    mask = nothot.iloc[:, 0].str.contains(r'\b(?:{})\b'.format('|'.join(words)))
    nothot = nothot[~mask]
    
    # checking whether the song is in both tables: 
    for index, row in hot.iterrows():
        df = nothot[(nothot['title'] != row['title']) & (nothot['artist'] != row['artist'])]
    
    return df.reset_index(drop=True)

In [7]:
NotHotSongs()

C:\Users\ameli\AppData\Local\Temp\ipykernel_6560\197547019.py:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  nothot.drop(['user_id', 'song_id', 'listen_count', 'song'], axis = 1, inplace = True)
C:\Users\ameli\AppData\Local\Temp\ipykernel_6560\197547019.py:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  nothot.drop_duplicates(inplace = True)
C:\Users\ameli\AppData\Local\Temp\ipykernel_6560\197547019.py:18: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  nothot[

,title,artist
0,Genie In A Bottle,Christina Aguilera
1,Swallowed In The Sea,Coldplay
2,We Ride,Rihanna
3,The Diary,Hollywood Undead
4,Your Star,The All-American Rejects
...,...,...
6137,Hummer,Foals
6138,Always Something There To Remind Me,Naked Eyes
6139,The Road To Mandalay,Robbie Williams
6140,Love Train,Wolfmother
